In [1]:
import pandas as pd
import numpy as np

In [2]:
# This will allow you to see all column names & rows when you are doing .head(). None of the column name will be truncated.
# source: https://stackoverflow.com/questions/49188960/how-to-show-all-of-columns-name-on-pandas-dataframe

pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [3]:
# source: https://gist.github.com/rozanecm/38f2901c592bdffc40726cb0473318cf
# Function which plays a beep of given duration and frequency.
# Useful for when executing things that need a while to finish, to get notified.
import os
def beep(duration = 1, freq = 1500):
    """ play tone of duration in seconds and freq in Hz. """
    os.system('play --no-show-progress --null --channels 1 synth %s sine %f' % (duration, freq))

In [4]:
train = pd.read_csv('../data/train.csv', dtype={'tipodepropiedad':'category', 'ciudad':'category', 'provincia':'category', 'id':'int32', 'antiguedad':'float16', 'habitaciones':'float16', 'garages':'float16', 'banos':'float16', 'metroscubiertos':'float16', 'metrostotales':'float16', 'idzona':'float16', 'lat':'float16', 'lng':'float16', 'gimnasio':'bool', 'usosmultiples':'bool', 'piscina':'bool', 'escuelascercanas':'bool', 'centroscomercialescercanos':'bool'}, parse_dates=['fecha'])
test = pd.read_csv('../data/test.csv', dtype={'tipodepropiedad':'category', 'ciudad':'category', 'provincia':'category', 'id':'int32', 'antiguedad':'float16', 'habitaciones':'float16', 'garages':'float16', 'banos':'float16', 'metroscubiertos':'float16', 'metrostotales':'float16', 'idzona':'float16', 'lat':'float16', 'lng':'float16', 'gimnasio':'bool', 'usosmultiples':'bool', 'piscina':'bool', 'escuelascercanas':'bool', 'centroscomercialescercanos':'bool'}, parse_dates=['fecha'])

***

## Analisis sobre la descripcion

In [5]:
train.shape

(240000, 23)

In [6]:
train_2 = train.copy()
train_2 = train_2[pd.notnull(train_2['descripcion'])]

In [7]:
pd.options.display.max_rows = 10
pd.options.display.max_colwidth = 10000
train_2.loc[train_2['descripcion'].str.contains('bosque'),'descripcion']

42                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

# Sobre viviendas:

Comodidades:
- Recamaras | Recamara
- Lavado
- Estacionamiento | Garage | Cochera
- Closet
- Sala comedor
- Comedor cocina
- Patio
- Vestidor
- Cuarto servicio
- Desayunadora
- Sala estudio
- Tanque estacionario
- Porton electrico
- Aire acondicionado

Area especial:
- Baño principal
- Baño vestidor
- Baño visita
- Areas verdes
- Roof garden
- Patio
- Cuarto lavado | Lavado cuarto | Area lavado
- Garage | Estacionamiento | Cochera
- Sala comedor
- Sala estudio
- Cocina

Areas verdes:
- Areas verdes
- Garden
- Patio
- Roof
- Jardin

Tiene seguridad: (bool)
- Porton electrico
- Caseta vigilancia | Vigilancia
- Acceso controlado

Palabras positivas:
- Excelente | Excelentes
- Bonita
- Lujo
- Hermosa
- Preciosa
- Completo
- Amplia

# Sobre comercial:

Palabras positivas:
- Ideal
- Doble

Tiene seguridad: (bool)
- Seguridad

Comodidades:
- Agua
- Vista

Areas verdes:
- Terraza

Cerca de otras entidades capitalistas: 
- Banco
- Gimnasio
- Metro
- Carretera
- Restaurante
- Plaza
- Parque
- Escuela

- Lago
- Playa

Area especial:
- Informes
- Terraza
- Recepcion
- Taller
- Almacen

Esta o es comercial: (bool)
- Comercial
- Comercio

Es oficina (bool)

Ubicado cerca o en esquina (bool)

Ubicado en avenida (bool)

Tiene bodega (bool)

Tiene planta alta - baja (bool)

# Sobre inversion:

Palabras positivas:
- Amplio
- Grande
- Hermoso
- Usos multiples

Areas verdes:
- Bosque
- Jardines
- Parque


Tiene servicio de algo (bool)

Tiene bodega (bool)

Areas entretenimiento: 
- Juegos infantiles
- club golf
- campo golf
- casa club
- Cancha tenis
- Colonia
- Club alberca

Es edificio (bool)

Es casa (bool)

Tiene seguridad: (bool)
- Barda perimetral
- Bardeado

Es parte de un lote (bool)

Esta en calle cerrada (bool)

Indica medidas frente y fondo (bool)

***

# Agregamos features en relacion a la descripcion

In [8]:
import unicodedata

def elimina_tildes(s):
    return ''.join((c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn'))

In [9]:
def contar_comodidades_en_descripcion(row):
    contador = 0
    x = elimina_tildes(str(row.descripcion))
    x = x.lower()
    if ((('recamaras') in x) | (('recamara') in x)):
        contador += 1.0
    if (('lavado') in x):
        contador += 1.0
    if ((('estacionamiento') in x) | (('garage') in x) | (('cochera') in x)):
        contador += 1.0
    if (('closet') in x):
        contador += 1.0
    if (('sala comedor') in x):
        contador += 1.0
    if (('comedor cocina') in x):
        contador += 1.0
    if (('patio') in x):
        contador += 1.0
    if (('vestidor') in x):
        contador += 1.0
    if (('cuarto servicio') in x):
        contador += 1.0
    if (('desayunadora') in x):
        contador += 1.0
    if ((('sala estudio') in x) | (('estudio sala') in x)):
        contador += 1.0
    if (('tanque estacionario') in x):
        contador += 1.0
    if (('porton electrico') in x):
        contador += 1.0
    if (('aire acondicionado') in x):
        contador += 1.0
    if (('agua') in x):
        contador += 1.0
    if (('vista') in x):
        contador += 1.0
    if (('elevadores') in x):
        contador += 1.0
    return contador

In [10]:
def contar_palabras_positivas(row):
    contador = 0
    x = elimina_tildes(str(row.descripcion))
    x = x.lower()
    if ((('bonita') in x) | (('bonito') in x)):
        contador += 1.0
    if (('lujo') in x):
        contador += 1.0
    if ((('hermosa') in x) | (('hermoso') in x)):
        contador += 1.0
    if ((('preciosa') in x) | (('precioso') in x)):
        contador += 1.0
    if ((('completo') in x) | (('completa') in x)):
        contador += 1.0
    if (('ideal') in x):
        contador += 1.0
    if (('doble') in x):
        contador += 1.0
    if ((('excelente') in x) | (('excelentes') in x)):
        contador += 1.0
    if ((('amplio') in x) | (('amplia') in x) | (('amplios') in x)):
        contador += 1.0
    if (('grande') in x):
        contador += 1.0
    if (('usos multiples') in x):
        contador += 1.0
    return contador

In [11]:
def contar_areas_dedicadas_en_descripcion(row):
    contador = 0
    x = elimina_tildes(str(row.descripcion))
    x = x.lower()
    if ((('cuarto lavado') in x) | (('lavado cuarto') in x) | (('area lavado') in x)):
        contador += 1.0
    if (('baño principal') in x):
        contador += 1.0
    if ((('estacionamiento') in x) | (('garage') in x) | (('cochera') in x)):
        contador += 1.0
    if (('baño vestidor') in x):
        contador += 1.0
    if (('sala comedor') in x):
        contador += 1.0
    if (('baño visita') in x):
        contador += 1.0
    if (('patio') in x):
        contador += 1.0
    if ((('sala estudio') in x) | (('estudio sala') in x)):
        contador += 1.0
    if (('cocina') in x):
        contador += 1.0
    if (('terraza') in x):
        contador += 1.0
    if (('informes') in x):
        contador += 1.0
    if (('recepcion') in x):
        contador += 1.0
    if (('taller') in x):
        contador += 1.0
    if (('almacen') in x):
        contador += 1.0
    return contador

In [12]:
def contar_areas_verdes_en_descripcion(row):
    contador = 0
    x = elimina_tildes(str(row.descripcion))
    x = x.lower()
    if (('patio') in x):
        contador += 1.0
    if ((('areas verdes') in x) | (('areas verde') in x)):
        contador += 1.0
    if (('roof') in x):
        contador += 1.0
    if (('garden') in x):
        contador += 1.0
    if ((('jardin') in x) | (('jardines') in x)):
        contador += 1.0
    if (('terraza') in x):
        contador += 1.0
    return contador

In [13]:
def contar_areas_entretenimiento(row):
    contador = 0
    x = elimina_tildes(str(row.descripcion))
    x = x.lower()
    if (('juegos infantiles') in x):
        contador += 1.0
    if ((('club golf') in x) | (('golf') in x)):
        contador += 1.0
    if (('campo golf') in x):
        contador += 1.0
    if (('casa club') in x):
        contador += 1.0
    if (('cancha tenis') in x):
        contador += 1.0
    if (('colonia') in x):
        contador += 1.0
    if (('club alberca') in x):
        contador += 1.0
    return contador

In [14]:
def contar_lugares_cerca(row):
    contador = 0
    x = elimina_tildes(str(row.descripcion))
    x = x.lower()
    if (('banco') in x):
        contador += 1.0
    if (('gimnasio') in x):
        contador += 1.0
    if (('metro') in x):
        contador += 1.0
    if (('carretera') in x):
        contador += 1.0
    if (('restaurante') in x):
        contador += 1.0
    if (('plaza') in x):
        contador += 1.0
    if (('parque') in x):
        contador += 1.0
    if (('escuela') in x):
        contador += 1.0
    if (('lago') in x):
        contador += 1.0
    if (('playa') in x):
        contador += 1.0
    return contador

In [15]:
def tiene_planta_alta(row):
    x = elimina_tildes(str(row.descripcion))
    x = x.lower()
    if (('planta alta') in x):
        return True
    else:
        return False

In [16]:
def tiene_planta_baja(row):
    x = elimina_tildes(str(row.descripcion))
    x = x.lower()
    if (('planta baja') in x):
        return True
    else:
        return False

In [17]:
def tiene_bodega(row):
    x = elimina_tildes(str(row.descripcion))
    x = x.lower()
    if (('bodega') in x):
        return True
    else:
        return False

In [18]:
def es_oficina(row):
    x = elimina_tildes(str(row.descripcion))
    x = x.lower()
    if (('oficina') in x):
        return True
    else:
        return False

In [19]:
def en_o_cerca_de_esquina(row):
    x = elimina_tildes(str(row.descripcion))
    x = x.lower()
    if (('esquina') in x):
        return True
    else:
        return False

In [20]:
def en_o_cerca_de_avenida(row):
    x = elimina_tildes(str(row.descripcion))
    x = x.lower()
    if (('avenida') in x):
        return True
    else:
        return False

In [21]:
def es_comercial(row):
    x = elimina_tildes(str(row.descripcion))
    x = x.lower()
    if ((('comercial') in x) | (('comercio') in x)):
        return True
    else:
        return False

In [22]:
def tiene_algun_tipo_de_servicio(row):
    x = elimina_tildes(str(row.descripcion))
    x = x.lower()
    if (('servicio') in x):
        return True
    else:
        return False

In [23]:
def es_edificio(row):
    x = elimina_tildes(str(row.descripcion))
    x = x.lower()
    if (('edificio') in x):
        return True
    else:
        return False

In [24]:
def es_casa(row):
    x = elimina_tildes(str(row.descripcion))
    x = x.lower()
    if (('casa') in x):
        return True
    else:
        return False

In [25]:
def es_parte_de_un_lote(row):
    x = elimina_tildes(str(row.descripcion))
    x = x.lower()
    if (('lote') in x):
        return True
    else:
        return False

In [26]:
def esta_en_calle_cerrada(row):
    x = elimina_tildes(str(row.descripcion))
    x = x.lower()
    if (('calle cerrada') in x):
        return True
    else:
        return False

In [27]:
def indica_medida_frente_y_fondo(row):
    x = elimina_tildes(str(row.descripcion))
    x = x.lower()
    if (('frente fondo') in x):
        return True
    else:
        return False

In [28]:
def usa_easybroker(row):
    x = elimina_tildes(str(row.descripcion))
    x = x.lower()
    if (('easybroker') in x):
        return True
    else:
        return False

In [29]:
def tiene_algun_tipo_de_seguridad(row):
    x = elimina_tildes(str(row.descripcion))
    x = x.lower()
    if ((('barda perimetral') in x) | \
        (('bardeado') in x) | \
        (('porton electrico') in x) | \
        (('vigilancia') in x) | \
        (('caseta vigilancia') in x) | \
        (('acceso controlado') in x) | \
        (('seguridad') in x)):
        return True
    else:
        return False

In [30]:
def add_desc_features(df):
    df['cant_comodidades_en_desc'] = df.apply(lambda x: contar_comodidades_en_descripcion(x), axis= 1)
    df['cant_palabras_positivas'] = df.apply(lambda x: contar_palabras_positivas(x), axis= 1)
    df['cant_areas_dedicadas'] = df.apply(lambda x: contar_areas_dedicadas_en_descripcion(x), axis= 1)
    df['cant_areas_verdes'] = df.apply(lambda x: contar_areas_verdes_en_descripcion(x), axis= 1)
    df['cant_areas_entretenimiento_cerca'] = df.apply(lambda x: contar_areas_entretenimiento(x), axis= 1)
    df['cant_lugares_cerca'] = df.apply(lambda x: contar_lugares_cerca(x), axis= 1)
    df['planta_alta'] = df.apply(lambda x: tiene_planta_alta(x), axis= 1)
    df['planta_baja'] = df.apply(lambda x: tiene_planta_baja(x), axis= 1)
    df['tiene_bodega'] = df.apply(lambda x: tiene_bodega(x), axis= 1)
    df['oficina'] = df.apply(lambda x: es_oficina(x), axis= 1)
    df['cerca_o_en_esquina'] = df.apply(lambda x: en_o_cerca_de_esquina(x), axis= 1)
    df['cerca_o_en_avenida'] = df.apply(lambda x: en_o_cerca_de_avenida(x), axis= 1)
    df['comercial'] = df.apply(lambda x: es_comercial(x), axis= 1)
    df['tiene_servicio'] = df.apply(lambda x: tiene_algun_tipo_de_servicio(x), axis= 1)
    df['edificio'] = df.apply(lambda x: es_edificio(x), axis= 1)
    df['casa'] = df.apply(lambda x: es_casa(x), axis= 1)
    df['parte_de_lote'] = df.apply(lambda x: es_parte_de_un_lote(x), axis= 1)
    df['calle_cerrada'] = df.apply(lambda x: esta_en_calle_cerrada(x), axis= 1)
    df['indica_frente_y_fondo'] = df.apply(lambda x: indica_medida_frente_y_fondo(x), axis= 1)
    df['usa_easybroker'] = df.apply(lambda x: usa_easybroker(x), axis= 1)
    df['tiene_seguridad'] = df.apply(lambda x: tiene_algun_tipo_de_seguridad(x), axis= 1)
    return df

In [31]:
from multiprocessing import  Pool

def parallelize_dataframe(df, func, n_cores):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [32]:
train = parallelize_dataframe(train, add_desc_features, 8)

In [33]:
test = parallelize_dataframe(test, add_desc_features, 8)

In [44]:
df_train = train.drop(train.columns[[0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22]], axis=1)

In [42]:
df_test = test.drop(test.columns[[0, 1, 2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21]], axis=1)

In [46]:
df_train.to_csv('../data/train_desc_features.csv',index=False,header=True)

In [45]:
df_test.to_csv('../data/test_desc_features.csv',index=False,header=True)

# * Fin agregado de features de descripcion *